In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/semantic_parsing/tree_slu/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%tensorflow_version 2.x
!pip install tensorflow-addons

TensorFlow 2.x selected.


In [3]:
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pprint
import logging
import time
import nltk

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (source, target_in, target_out)

In [0]:
def dataset(is_training, params):
  _shapes = ([None], [None], [None])
  _types = (tf.int32, tf.int32, tf.int32)
  _pads = (0, 0, 0)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['train_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['eval_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class Embed(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.embedding = tf.Variable(np.load('../vocab/word.npy'),
                                 dtype=tf.float32,
                                 name='pretrained_embedding')
  
  def call(self, inputs):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    return x

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
      params['rnn_units'], return_state=True, return_sequences=True))
    self.state_fc = tf.keras.layers.Dense(params['rnn_units'], params['activation'], name='state_fc')
  
  def call(self, inputs, training):
    x = self.dropout(inputs, training=training)
    
    encoder_o, state_fw_h, state_fw_c, state_bw_h, state_bw_c = self.encoder(x)
    encoder_s = [
      self.state_fc(tf.concat((state_fw_h, state_bw_h), -1)),
      self.state_fc(tf.concat((state_fw_c, state_bw_c), -1)),]
    
    return encoder_o, encoder_s

In [0]:
class TiedDense(tf.keras.layers.Layer):
  def __init__(self, tied_embed, out_dim):
    super().__init__()
    self.tied_embed = tied_embed
    self.out_dim = out_dim
  
  def build(self, input_shape):
    self.bias = self.add_weight(name='bias',
                                shape=[self.out_dim],
                                trainable=True)
    """
    self.proj_W = self.add_weight(name='proj_W',
                                  shape=[2*params['rnn_units'], params['embed_dim']],
                                  trainable=True)
    self.proj_b = self.add_weight(name='proj_b',
                                  shape=[params['embed_dim']],
                                  trainable=True)
    """
    super().build(input_shape)
  
  @tf.function
  def call(self, inputs):
    """
    inputs = params['activation'](inputs @ self.proj_W + self.proj_b)
    """
    x = tf.matmul(inputs, self.tied_embed, transpose_b=True)
    x += self.bias
    return x
  
  def compute_output_shape(self, input_shape):
    return input_shape[:-1].concatenate(self.out_dim)

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.embed = Embed()

    self.encoder = Encoder(params)

    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])

    self.attn = tfa.seq2seq.BahdanauAttention(params['rnn_units'])

    self.decoder_cell = tfa.seq2seq.AttentionWrapper(
      tf.keras.layers.LSTMCell(params['rnn_units']),
      self.attn,
      attention_layer_size=params['rnn_units'])
    
    self.proj_layer = TiedDense(self.embed.embedding, len(params['tgt2idx'])+1)

    self.teach_forcing = tfa.seq2seq.BasicDecoder(
      self.decoder_cell,
      tfa.seq2seq.sampler.TrainingSampler(),
      output_layer = self.proj_layer)

    self.beam_search = tfa.seq2seq.BeamSearchDecoder(
      self.decoder_cell,
      beam_width = params['beam_width'],
      embedding_fn = lambda x: self.embed(x),
      output_layer = self.proj_layer,
      maximum_iterations = 80,)

  
  def call(self, inputs, training=True):
    if training:
      source, target_in = inputs
    else:
      source = inputs
    batch_sz = tf.shape(source)[0]

    encoder_o, encoder_s = self.encoder(self.embed(source), training=training)

    if training:
      self.attn([encoder_o, tf.math.count_nonzero(source, 1)], setup_memory=True)
      attn_state = self.decoder_cell.get_initial_state(batch_size=batch_sz, dtype=tf.float32)
      attn_state = attn_state.clone(cell_state=encoder_s)

      decoder_o, _, _ = self.teach_forcing(
        inputs = self.dropout(self.embed(target_in), training=training),
        initial_state = attn_state,
        sequence_length = tf.math.count_nonzero(target_in, 1, dtype=tf.int32))

      logits_or_ids = decoder_o.rnn_output
    else:
      encoder_o_t = tfa.seq2seq.tile_batch(encoder_o, params['beam_width'])
      encoder_len_t = tfa.seq2seq.tile_batch(tf.math.count_nonzero(source, 1), params['beam_width'])
      encoder_s_t = tfa.seq2seq.tile_batch(encoder_s, params['beam_width'])

      self.attn([encoder_o_t, encoder_len_t], setup_memory=True)
      attn_state = self.decoder_cell.get_initial_state(batch_size=batch_sz*params['beam_width'], dtype=tf.float32)
      attn_state = attn_state.clone(cell_state=encoder_s_t)

      decoder_o, _, _ = self.beam_search(
        None,
        start_tokens = tf.tile(tf.constant([1], tf.int32), [batch_sz]),
        end_token = 2,
        initial_state = attn_state,)

      logits_or_ids = decoder_o.predicted_ids[:, :, 0]

    return logits_or_ids

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True

In [0]:
params = {
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model/',
    'dropout_rate': 0.2,
    'rnn_units': 300,
    'embed_dim': 300,
    'activation': tf.nn.relu,
    'beam_width': 5,
    'lr': 4e-4,
    'clip_norm': .1,
    'buffer_size': 31279,
    'train_batch_size': 32,
    'eval_batch_size': 128,
    'num_patience': 5,
}

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [14]:
model = Model(params)
model.build(input_shape=[[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

[('pretrained_embedding:0', TensorShape([8692, 300])),
 ('encoder/bidirectional/forward_lstm/kernel:0', TensorShape([300, 1200])),
 ('encoder/bidirectional/forward_lstm/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('encoder/bidirectional/forward_lstm/bias:0', TensorShape([1200])),
 ('encoder/bidirectional/backward_lstm/kernel:0', TensorShape([300, 1200])),
 ('encoder/bidirectional/backward_lstm/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('encoder/bidirectional/backward_lstm/bias:0', TensorShape([1200])),
 ('encoder/state_fc/kernel:0', TensorShape([600, 300])),
 ('encoder/state_fc/bias:0', TensorShape([300])),
 ('BahdanauAttention/attention_v:0', TensorShape([300])),
 ('attention_wrapper/BahdanauAttention/kernel:0', TensorShape([300, 300])),
 ('BahdanauAttention/kernel:0', TensorShape([600, 300])),
 ('attention_wrapper/attention_layer/kernel:0', TensorShape([900, 300])),
 ('attention_wrapper/lstm_cell_3/kernel:0', TensorShape([600, 1200])),
 ('attention_wrapper/lstm_cell_3

In [0]:
decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

In [0]:
history_acc = []
best_acc = .0

In [0]:
t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.propagate = False
logger.setLevel(logging.INFO)

In [0]:
def minimal_test(model, params):
  test_str = ['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']
  test_arr = tf.convert_to_tensor([[params['tgt2idx'][w] for w in test_str]])
  generated = model(inputs=test_arr, training=False)

  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str))
  parsed = ' '.join([params['idx2tgt'][idx] for idx in generated[0].numpy()])
  parsed = parsed.replace('<end>', '').strip()
  print('parsed:', parsed)
  print()
  try:
    nltk.tree.Tree.fromstring(parsed.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
while True:
  # TRAINING
  is_training = True
  for i, (source, target_in, target_out) in enumerate(dataset(is_training=is_training, params=params)):
    with tf.GradientTape() as tape:
      logits_or_ids = model((source, target_in), training=is_training)
      
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=target_out, logits=logits_or_ids)
      weights = tf.cast(tf.sign(target_in), tf.float32)
      loss = tf.reduce_sum(loss * weights) / tf.reduce_sum(weights)

    variables = model.trainable_variables
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
        global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    
    global_step += 1

  # EVALUATION
  is_training = False
  minimal_test(model, params)
  m = tf.keras.metrics.Mean()
  
  for i, (source, target_in, target_out) in enumerate(dataset(is_training=is_training, params=params)):
    generated = model(inputs=source, training=is_training)
    seq_lens = tf.argmax(tf.cast(tf.equal(target_out, 2), tf.int32), axis=1)
    for pred, tgt, seq_len in zip(generated.numpy(), target_out.numpy(), seq_lens.numpy()):
      matched = np.all(pred[:seq_len+1] == tgt[:seq_len+1])
      m.update_state(int(matched))
  
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Exact Match Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

Reading ../data/train.tsv
INFO:tensorflow:Step 0 | Loss: 9.2826 | Spent: 6.0 secs | LR: 0.000400
INFO:tensorflow:Step 50 | Loss: 3.4936 | Spent: 31.0 secs | LR: 0.000400
INFO:tensorflow:Step 100 | Loss: 2.5489 | Spent: 31.4 secs | LR: 0.000400
INFO:tensorflow:Step 150 | Loss: 1.8968 | Spent: 30.4 secs | LR: 0.000399
INFO:tensorflow:Step 200 | Loss: 1.5225 | Spent: 31.3 secs | LR: 0.000399
INFO:tensorflow:Step 250 | Loss: 1.5356 | Spent: 31.6 secs | LR: 0.000399
INFO:tensorflow:Step 300 | Loss: 1.1140 | Spent: 34.1 secs | LR: 0.000399
INFO:tensorflow:Step 350 | Loss: 0.9405 | Spent: 32.0 secs | LR: 0.000399
INFO:tensorflow:Step 400 | Loss: 0.6781 | Spent: 31.5 secs | LR: 0.000398
INFO:tensorflow:Step 450 | Loss: 0.6539 | Spent: 31.1 secs | LR: 0.000398
INFO:tensorflow:Step 500 | Loss: 0.7228 | Spent: 31.4 secs | LR: 0.000398
INFO:tensorflow:Step 550 | Loss: 0.5562 | Spent: 32.1 secs | LR: 0.000398
INFO:tensorflow:Step 600 | Loss: 0.5013 | Spent: 31.4 secs | LR: 0.000398
INFO:tensorflow:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Evaluation: Testing Exact Match Accuracy: 0.383
INFO:tensorflow:Best Accuracy: 0.383
Reading ../data/train.tsv
INFO:tensorflow:Step 1000 | Loss: 0.3383 | Spent: 167.3 secs | LR: 0.000396
INFO:tensorflow:Step 1050 | Loss: 0.3575 | Spent: 31.2 secs | LR: 0.000396
INFO:tensorflow:Step 1100 | Loss: 0.3602 | Spent: 33.0 secs | LR: 0.000396
INFO:tensorflow:Step 1150 | Loss: 0.2549 | Spent: 32.3 secs | LR: 0.000395
INFO:tensorflow:Step 1200 | Loss: 0.2287 | Spent: 31.8 secs | LR: 0.000395
INFO:tensorflow:Step 1250 | Loss: 0.3000 | Spent: 32.4 secs | LR: 0.000395
INFO:tensorflow:Step 1300 | Loss: 0.2924 | Spent: 31.4 secs | LR: 0.000395
INFO:tensorflow:Step 1350 | Loss: 0.1161 | Spent: 31.1 secs | LR: 0.000395
INFO:tensorflow:Step 1400 | Loss: 0.1842 | Spent: 32.1 secs | LR: 0.000394
INFO:tensorflow:Step 1450 | Loss: 0.2146 | Spent: 31.8 secs | LR: 0.000394
INFO:tensorflow:Step 1500 | Loss: 0.2259 | Spent: 30.4 secs | LR: 0.000394
INFO:tensorflow:Step 1550 | Loss: 0.1540 | Spen

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Evaluation: Testing Exact Match Accuracy: 0.597
INFO:tensorflow:Best Accuracy: 0.597
Reading ../data/train.tsv
INFO:tensorflow:Step 2000 | Loss: 0.2686 | Spent: 186.5 secs | LR: 0.000392
INFO:tensorflow:Step 2050 | Loss: 0.1794 | Spent: 31.5 secs | LR: 0.000392
INFO:tensorflow:Step 2100 | Loss: 0.0995 | Spent: 30.3 secs | LR: 0.000392
INFO:tensorflow:Step 2150 | Loss: 0.2179 | Spent: 31.6 secs | LR: 0.000391
INFO:tensorflow:Step 2200 | Loss: 0.1103 | Spent: 30.6 secs | LR: 0.000391
INFO:tensorflow:Step 2250 | Loss: 0.0681 | Spent: 31.2 secs | LR: 0.000391
INFO:tensorflow:Step 2300 | Loss: 0.1054 | Spent: 30.5 secs | LR: 0.000391
INFO:tensorflow:Step 2350 | Loss: 0.1282 | Spent: 30.8 secs | LR: 0.000391
INFO:tensorflow:Step 2400 | Loss: 0.0614 | Spent: 32.4 secs | LR: 0.000390
INFO:tensorflow:Step 2450 | Loss: 0.0933 | Spent: 32.1 secs | LR: 0.000390
INFO:tensorflow:Step 2500 | Loss: 0.0860 | Spent: 32.1 secs | LR: 0.000390
INFO:tensorflow:Step 2550 | Loss: 0.1360 | Spen

...

INFO:tensorflow:Evaluation: Testing Exact Match Accuracy: 0.724
INFO:tensorflow:Best Accuracy: 0.724
Reading ../data/train.tsv
INFO:tensorflow:Step 21550 | Loss: 0.0101 | Spent: 181.9 secs | LR: 0.000322
INFO:tensorflow:Step 21600 | Loss: 0.0104 | Spent: 31.7 secs | LR: 0.000322
INFO:tensorflow:Step 21650 | Loss: 0.0003 | Spent: 30.2 secs | LR: 0.000322
INFO:tensorflow:Step 21700 | Loss: 0.0045 | Spent: 31.4 secs | LR: 0.000322
INFO:tensorflow:Step 21750 | Loss: 0.0156 | Spent: 30.7 secs | LR: 0.000321
INFO:tensorflow:Step 21800 | Loss: 0.0074 | Spent: 30.6 secs | LR: 0.000321
INFO:tensorflow:Step 21850 | Loss: 0.0155 | Spent: 30.8 secs | LR: 0.000321
INFO:tensorflow:Step 21900 | Loss: 0.0025 | Spent: 31.5 secs | LR: 0.000321
INFO:tensorflow:Step 21950 | Loss: 0.0035 | Spent: 30.9 secs | LR: 0.000321
INFO:tensorflow:Step 22000 | Loss: 0.0066 | Spent: 32.5 secs | LR: 0.000321
INFO:tensorflow:Step 22050 | Loss: 0.0020 | Spent: 32.0 secs | LR: 0.000320
INFO:tensorflow:Step 22100 | Loss: 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Evaluation: Testing Exact Match Accuracy: 0.724
INFO:tensorflow:Best Accuracy: 0.724
Reading ../data/train.tsv
INFO:tensorflow:Step 22500 | Loss: 0.0114 | Spent: 181.2 secs | LR: 0.000319
INFO:tensorflow:Step 22550 | Loss: 0.0113 | Spent: 31.4 secs | LR: 0.000319
INFO:tensorflow:Step 22600 | Loss: 0.0219 | Spent: 32.7 secs | LR: 0.000319
INFO:tensorflow:Step 22650 | Loss: 0.0267 | Spent: 31.4 secs | LR: 0.000319
INFO:tensorflow:Step 22700 | Loss: 0.0020 | Spent: 31.2 secs | LR: 0.000318
INFO:tensorflow:Step 22750 | Loss: 0.0018 | Spent: 31.6 secs | LR: 0.000318
INFO:tensorflow:Step 22800 | Loss: 0.0011 | Spent: 29.9 secs | LR: 0.000318
INFO:tensorflow:Step 22850 | Loss: 0.0020 | Spent: 30.9 secs | LR: 0.000318
INFO:tensorflow:Step 22900 | Loss: 0.0208 | Spent: 31.2 secs | LR: 0.000318
INFO:tensorflow:Step 22950 | Loss: 0.0025 | Spent: 30.6 secs | LR: 0.000318
INFO:tensorflow:Step 23000 | Loss: 0.0074 | Spent: 31.5 secs | LR: 0.000317
INFO:tensorflow:Step 23050 | Loss: 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Evaluation: Testing Exact Match Accuracy: 0.724
INFO:tensorflow:Best Accuracy: 0.724
Reading ../data/train.tsv
INFO:tensorflow:Step 23500 | Loss: 0.0013 | Spent: 179.2 secs | LR: 0.000316
INFO:tensorflow:Step 23550 | Loss: 0.0033 | Spent: 30.2 secs | LR: 0.000316
INFO:tensorflow:Step 23600 | Loss: 0.0050 | Spent: 31.8 secs | LR: 0.000316
INFO:tensorflow:Step 23650 | Loss: 0.0073 | Spent: 31.1 secs | LR: 0.000315
INFO:tensorflow:Step 23700 | Loss: 0.0042 | Spent: 31.7 secs | LR: 0.000315
INFO:tensorflow:Step 23750 | Loss: 0.0014 | Spent: 31.2 secs | LR: 0.000315
INFO:tensorflow:Step 23800 | Loss: 0.0015 | Spent: 30.6 secs | LR: 0.000315
INFO:tensorflow:Step 23850 | Loss: 0.0025 | Spent: 31.9 secs | LR: 0.000315
INFO:tensorflow:Step 23900 | Loss: 0.0112 | Spent: 30.5 secs | LR: 0.000315
INFO:tensorflow:Step 23950 | Loss: 0.0053 | Spent: 31.2 secs | LR: 0.000314
INFO:tensorflow:Step 24000 | Loss: 0.0061 | Spent: 32.0 secs | LR: 0.000314
INFO:tensorflow:Step 24050 | Loss: 0